<a href="https://colab.research.google.com/github/neesmusuns/vigenere/blob/master/vigenere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#!pip install langdetect
#!pip install langid
# Call this to install required packages

from collections import Counter
from itertools import chain
from functools import reduce
import string
import re
import itertools
import langdetect
from langid.langid import LanguageIdentifier, model

ALPHABET ='abcdefghijklmnopqrstuvwxyz'
ETAOIN = 'etaoinsrhdlucmfywgpbvkxqjz' # Alphabet ordered by frequency (most frequent first)
NUM_LETTERS = len(ALPHABET) # length of our alphabet
language_identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)


def prepare_string(s):
  p = ''.join(filter(str.isalpha, s)).lower()
  return p, len(p)

def find_factors(n):
  """Find factors for an integer n"""    
  return reduce(list.__add__, ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0))


def friedman_test(ciphertext):
  """A function that performs the Friedman test on a given ciphertext"""
  ciphertext, n = prepare_string(ciphertext)
  cipher_f = dict(Counter(ciphertext)) # frequencies in ciphertext
  alpha_f = dict.fromkeys(string.ascii_lowercase, 0) # frequencies in alphabet
  for letter, count in cipher_f.items():
    if letter in alpha_f:
      if(alpha_f[letter] == 0):
        alpha_f[letter] = count
  f_sum = 0
  for f in alpha_f.values():
    f_sum += f * (f-1)
  index = f_sum/(n*(n-1)) # index of coincidence
  key_length = (0.0265*n)/(0.065-index + n*(index-0.0385))
  key_length = int(round(key_length))
  return index, key_length


def kasiski_examination(ciphertext):
  """A function that performs kasiski examination on a given ciphertext"""
  ciphertext, n = prepare_string(ciphertext)
  recurring_strings = {}
  # find occurrences of substrings of length 3,4,5
  for k in range(3,6):
    for i in range(n-k): 
      substr = ciphertext[i:i+k]
      j = [m.start() for m in re.finditer(substr, ciphertext)]
      recurring_strings.update( {substr : j} )
  spaces, factors = [], []
  for key in recurring_strings: # find spaces between occurrences
    if(len(recurring_strings[key]) > 1):
      for i in range(len(recurring_strings[key])):        
        spaces.append(abs(recurring_strings[key][i]-recurring_strings[key][i-1]))
  for space in spaces:
    factors.append(find_factors(space))
  factors = list(chain.from_iterable(factors))
  # sort by frequency starting with most frequent
  key_lengths = [i for items, c in Counter(factors).most_common() for i in [items] * c]
  key_lengths = list(dict.fromkeys(key_lengths))
  return key_lengths[0:3]

def vigenere_encode(plaintext, key):
  """Encodes a plaintext message with vigenere encoding, given a key"""
  plaintext, n = prepare_string(plaintext)
  key, kn = prepare_string(key)
  ciphertext = ""
  for i in range(n):
    ciphertext += chr((ord(plaintext[i]) + ord(key[i%kn]) -97*2) % 26 + 97)
  return ciphertext

def vigenere_decode(ciphertext, key):
  """Decodes a ciphertext message with vigenere encoding, given a key"""
  ciphertext, n = prepare_string(ciphertext)
  key, kn = prepare_string(key)
  plaintext = ""
  for i in range(n):
    plaintext += chr((ord(ciphertext[i]) - ord(key[i%kn])) % 26 + 97)
  return plaintext

def getLetterCount(message):
  """Returns a dictionary with single letter keys and the frequency at which
  they appear in the message
  """
  letterCount = {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0,
                 'i': 0, 'j': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'o': 0, 'p': 0,
                 'q': 0, 'r': 0, 's': 0, 't': 0, 'u': 0, 'v': 0, 'w': 0, 'x': 0,
                 'y': 0, 'z': 0}

  for letter in message.lower():
    if letter in ALPHABET:
      letterCount[letter] += 1

  return letterCount


def getItemAtIndexZero(x):
  """Get the first element of an array"""
  return x[0]

def getItemAtIndexOne(x):
  """Get the second element of an array"""
  return x[1]

def getFrequencyOrder(message):
  """Get a string of the letters in order of their frequency in the message"""

  letterToFreq = getLetterCount(message)
  
  freqToLetter = {}
  for letter in ALPHABET:
    if letterToFreq[letter] not in freqToLetter:
      freqToLetter[letterToFreq[letter]] = [letter]
    else:
      freqToLetter[letterToFreq[letter]].append(letter)

  for freq in freqToLetter:
    freqToLetter[freq].sort(key=ETAOIN.find, reverse=True)
    freqToLetter[freq] = ''.join(freqToLetter[freq])

  freqPairs = list(freqToLetter.items())
  freqPairs.sort(key=getItemAtIndexZero, reverse=True)
    
  freqOrder = []
  for freqPair in freqPairs:
    freqOrder.append(freqPair[1])
    
  return ''.join(freqOrder)

def englishFreqMatchScore(message):
  """
  Get the number of matches that the string in the message
  parameter has when its letter frequency is compared to English
  letter frequency.
  """
  freqOrder = getFrequencyOrder(message)

  matchScore = 0
  # Look at matches for most common letters
  for commonLetter in ETAOIN[:6]:
    if commonLetter in freqOrder[:6]:
      matchScore += 1
  # Look at matches for least common letters
  for uncommonLetter in ETAOIN[-6:]:
    if uncommonLetter in freqOrder[-6:]:
      matchScore += 1

  return matchScore

pattern = re.compile('[^a-z]')

def getNthLetters(n, keyLen, message):
  """Get the Nth leter for every KeyLen in the message"""
  message = pattern.sub('', message)
    
  i = n - 1
  l = []
  while i < len(message):
    l.append(message[i])
    i += keyLen
  return ''.join(l)
    
def hackKeyLen(ciphertext, likeliestKeyLength):
  """Find the likeliest keys for the key length"""
  allLetterFreqs = []
  for nth in range(1, likeliestKeyLength + 1):
    nthLetters = getNthLetters(nth, likeliestKeyLength, ciphertext)
    
    letterFreqs = []
    for possibleKey in ALPHABET:
      decryptedText = vigenere_decode(nthLetters, possibleKey)
      keyAndFreqMatchTuple = (possibleKey, englishFreqMatchScore(decryptedText))
      letterFreqs.append(keyAndFreqMatchTuple)
      # Sort by match score
    letterFreqs.sort(key=getItemAtIndexOne, reverse=True)
    
    allLetterFreqs.append(letterFreqs[:NUM_LETTERS])
    
  count = 0
  # Try all combinations of the likeliest letters for all positions in the key
  for indexes in itertools.product(range(NUM_LETTERS), repeat=likeliestKeyLength):
    # Generate a key from the letters in allLetterFreqs
    possibleKey = ''
    for i in range(likeliestKeyLength):
      possibleKey += allLetterFreqs[i][indexes[i]][0]
    
    decryptedText = vigenere_decode(ciphertext, possibleKey)
    
    langdetect_lang = langdetect.detect_langs(decryptedText)
    langid_lang = language_identifier.classify(decryptedText)
    count += 1
    if count % 100 == 0:
      print("Attempt", count, ":", langid_lang, "for key:", possibleKey)
      
    for detected_lang in langdetect_lang:
      if detected_lang.lang == 'en' and detected_lang.prob > 0.9 and langid_lang[0] == 'en' and langid_lang[1] > 0.5:
        # Let user manually confirm that the key is correct
        print('Found key:', possibleKey)
        print('Truncated output:', decryptedText[:50], "...")
        print()
        print('Input Y if this is the expected result or press enter to continue:')
        input_command = input('>> ')
    
        if input_command.strip().upper() == 'Y':
          return decryptedText, True
  # If we find no decryption that might be English we return None
  return None


print("""The Vigenère decoder will decrypt your message. Here's an example of an 
encrypted message: \n'luehghrsghgwleasagidfjahunrgardxfgopdnszsnnsluelwfttjacpfndxsattje
iigeitkqugaagizrkagadxcrgddqrjkuouubnkwatxgaaadrakwaicyfsjuuahqrahl
nnstnkxftsdvnwtjrmjuultkfrtdvaqdricluergadxlvocksarwqbnlueejbsewptd
jfemhrrxwactvzicwespfqoizrrhwgtawesujrqjwataqpagjvesscojuuoukgaglrr
taghtjnrdmadizrigfrczgeocsoealghtkrwtjrfxwectdlgjsedtvgozwrpujbmujr
eoaagwgjekwefgwrzxftddwfndlxiadnsdmeddmthhlnriweemurshaiewwntsgrsdd
qhpfqsrszeigoersyltvfojjqojyusplrrbluaiafsiaylphclxwqtdsaypdnszsaog
cyocvvktgydiazegluehatnxxvcpfpedxghtfvczfnmtknshgpiplvocovtwqhkdfpu
alhrtonsxezoglnlxrrdxfghtoeiiaaghgsrdtrrikrrkapeesetxuhlpjyywafcddy
erlvocgssdftsdxnsdmeddmth'""")
c = input("Enter your encrypted message: ") 
"""
message = The sourdough tradition was carried into Alaska and the western Canadian 
              territories during the Klondike Gold Rush of 1898. Conventional leavenings 
              such as yeast and baking soda were much less reliable in the conditions 
              faced by the prospectors. Experienced miners and other settlers frequently 
              carried a pouch of starter either around their neck or on a belt; these 
              were fiercely guarded to keep from freezing. However, freezing does not 
              kill a sourdough starter; excessive heat does. Old hands came to be called 
              "sourdoughs", a term that is still applied to any Alaskan or Klondike old-timer. 
              The significance of the nickname's association with Yukon culture was 
              immortalized in the writings of Robert Service, particularly his collection 
             of "Songs of a Sourdough".
"""

# c = vigenere_encode(message, "snap")
# print("Encoded message:", c)
print("Results:")
key_lengths = [0,0,0,0]
index, key_lengths[0] = friedman_test(c)
key_lengths[1], key_lengths[2], key_lengths[3] = kasiski_examination(c)
key_lengths = list(dict.fromkeys(key_lengths)) # remove duplicates
print("Friedman test: index of coincidence: {:.3f}".format(index), ", possible key length:", key_lengths[0])
print("Kasiski examination: possible key lengths:", key_lengths[1], key_lengths[2])

for keyLen in key_lengths:
  text, response = hackKeyLen(c, keyLen)
  if(response == True):
    print("Decrypted text:", text)
    break


The Vigenère decoder will decrypt your message. Here's an example of an 
encrypted message: 
'luehghrsghgwleasagidfjahunrgardxfgopdnszsnnsluelwfttjacpfndxsattje
iigeitkqugaagizrkagadxcrgddqrjkuouubnkwatxgaaadrakwaicyfsjuuahqrahl
nnstnkxftsdvnwtjrmjuultkfrtdvaqdricluergadxlvocksarwqbnlueejbsewptd
jfemhrrxwactvzicwespfqoizrrhwgtawesujrqjwataqpagjvesscojuuoukgaglrr
taghtjnrdmadizrigfrczgeocsoealghtkrwtjrfxwectdlgjsedtvgozwrpujbmujr
eoaagwgjekwefgwrzxftddwfndlxiadnsdmeddmthhlnriweemurshaiewwntsgrsdd
qhpfqsrszeigoersyltvfojjqojyusplrrbluaiafsiaylphclxwqtdsaypdnszsaog
cyocvvktgydiazegluehatnxxvcpfpedxghtfvczfnmtknshgpiplvocovtwqhkdfpu
alhrtonsxezoglnlxrrdxfghtoeiiaaghgsrdtrrikrrkapeesetxuhlpjyywafcddy
erlvocgssdftsdxnsdmeddmth'
Enter your encrypted message: luehghrsghgwleasagidfjahunrgardxfgopdnszsnnsluelwfttjacpfndxsattje iigeitkqugaagizrkagadxcrgddqrjkuouubnkwatxgaaadrakwaicyfsjuuahqrahl nnstnkxftsdvnwtjrmjuultkfrtdvaqdricluergadxlvocksarwqbnlueejbsewptd jfemhrrxwactvzicwespfqoizrrhwgtawes